In [ ]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head()

# 3/13  2. Базовые операции со столбцами DataFrame 

СОЗДАНИЕ КОПИИ ТАБЛИЦЫ

In [ ]:
melb_df = melb_data.copy()
melb_df.head()

УДАЛЕНИЕ СТОЛБЦОВ

За удаление строк и столбцов в таблице отвечает метод `drop()`.  

Удалим столбцы `index` и `Coordinates` из таблицы с помощью метода `drop()`. Выведем первые пять строк таблицы и убедимся, что всё прошло успешно.

Основные параметры метода `.drop()`

`labels` — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;  
`axis` — ось совершения операции, `axis=0` — удаляются строки, `axis=1` — удаляются столбцы;  
`inplace` — если параметр выставлен на `True`, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на `False` — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен `False`.

In [ ]:
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)# создание копии с изменениями
melb_df.head()

# Альтернативный вариант:

# melb_df.drop(['index','Coordinates'],axis=1,inplace=True)# меняет объект
# melb_df.head()

МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

→ `Pandas` поддерживает базовые математические операции между столбцами: **столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень**. С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

Например, давайте создадим переменную `total_rooms`, в которой будем хранить общее количество комнат в здании. Для этого выполним сложение столбцов с количеством комнат, ванн и спален:

In [ ]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

А теперь введём признак `MeanRoomsSquare`, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат:

In [ ]:
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

Можно ввести ещё один интересный признак — `AreaRatio`, коэффициент соотношения площади здания (`BuildingArea`) и площади участка (`Landsize`). Для этого разницу двух площадей поделим на их сумму:

In [ ]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

Что показывает такой коэффициент? Если присмотреться, можно увидеть, что `AreaRatio` лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.  
Если площадь участка больше площади здания, то коэффициент начинает расти в отрицательную сторону, и в положительную сторону, если наоборот.  
Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1. Если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.  
Таким образом, значение в столбце `AreaRatio` служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

# 4/13  3. Работа с датами в DataFrame

ФОРМАТ DATETIME

Единым способом обозначения даты и времени в Pandas является формат `datetime`, который записывается как `YYYY-MM-DD HH: MM: SS`, то есть составляющие времени указываются в следующем порядке: `год, месяц, день, час, минута, секунда`. 

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [ ]:
display(melb_df['Date'])

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат `datetime`, можно воспользоваться функцией `pandas.to_datetime()`. В нашем случае в функции нужно указать параметр `dayfirst=True`, который будет обозначать, что в первоначальном признаке первым идёт день. Преобразуем столбец `Date` в формат `datetime`, передав его в эту функцию:

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных `datetime` позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

`date` — дата;  
`year`, `month`, `day` — год, месяц, день;  
`time` — время;  
`hour`, `minute`, `second` — час, минута, секунда;  
`dayofweek` — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;  
`day_name` — название дня недели;  
`dayofyear` — порядковый день года;  
`quarter` — квартал (интервал в три месяца).  

Аксессор — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью `pd.to_datetime()`.

Обратите внимание, что вы не сможете обратиться к аксессору, если ваш столбец не приведён к типу `datetime`.

Например, обратившись по атрибуту `dt.year` в столбце `Date`, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [ ]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])